# Demo of VAE for anomaly detection of diverse sensor data
Trained on non-fault data to learn a normal expectation.

Build the model for 512 time points per cycle with 14 sensors.

In [1]:
%matplotlib inline

In [2]:
from IPython.core.debugger import set_trace

In [3]:
from demo import *
device

device(type='cuda', index=0)

In [4]:
desc

'accumulator'

In [5]:
model = VAE1D(size, n_channels, n_latent)
model = model.to(device)
print(f"Time-series size = {size}, number of sensors = {n_channels}, "
      f"latent size = {n_latent}")

Time-series size = 512, number of sensors = 14, latent size = 50


Load the best parameters from training.

In [6]:
model = load_checkpoint(model, device)

Checkpoint Performance:
Validation loss: 14.268
Epoch: 210


In [7]:
params = model.demo()

Input size: torch.Size([1, 14, 512])
Encoded size: torch.Size([1, 1024, 4])
Latent size: torch.Size([1, 50, 1])
Decoded (output) size: torch.Size([1, 14, 512])


In [8]:
X, E, L, D = [param.cpu().detach().numpy() for param in params]
MSE = np.power((X - D), 2).sum() / 2
print(f"MSE of random data = {MSE:.3f}")

MSE of random data = 666.951


In [9]:
X[:, :4, :4]

array([[[0.03756665, 0.39188507, 0.679316  , 0.6438357 ],
        [0.8571021 , 0.16819857, 0.9926153 , 0.633139  ],
        [0.14229044, 0.24964722, 0.15595786, 0.7348866 ],
        [0.7716662 , 0.80065954, 0.3989681 , 0.01456177]]], dtype=float32)

In [10]:
D[:, :4, :4]

array([[[2.1584080e-01, 2.1162044e-01, 2.3755118e-01, 4.0655178e-01],
        [3.6056536e-01, 1.4748839e-01, 6.9426149e-02, 2.3096055e-04],
        [6.4407456e-01, 5.3543776e-01, 5.1022458e-01, 4.0813673e-01],
        [2.2680111e-01, 3.4148991e-01, 3.6603764e-01, 6.2843657e-01]]],
      dtype=float32)

Next we need to load the datasets of sensors readings.

In [11]:
data_path = Path(f'data/hydraulic/{desc}')
train_dl, val_dl, test_dl = load_datasets(data_path)

In [12]:
print(len(train_dl), len(val_dl), len(test_dl))

27 7 1157


Let's get some random samples from the dataset for examples and generate new versions.

In [13]:
data, targets = get_random_samples(test_dl)

In [14]:
classes = list_target_classes(test_dl)

0 = fail
1 = norm


In [15]:
targets = targets.cpu().numpy()
targets[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [16]:
print('normals = ', targets.sum())

normals =  349


In [17]:
print('failures = ', len(targets) - targets.sum())

failures =  808


In [18]:
criterion = VAE1DLoss()
criterion = criterion.to(device)

In [19]:
model.eval()
with torch.no_grad():
    # Generate
    data = data.to(device)
    gen_data, mu, logvar = model(data)
    loss, loss_desc = criterion(gen_data, data, mu, logvar, reduce=False)
    

In [20]:
gen_err = -loss_desc['logp']
gen_err[:10]

tensor([ 1.1756,  3.2176, 16.6307,  1.3021,  8.2106,  3.6154,  3.1698,  2.0904,
         2.7066, 11.7679], device='cuda:0')

In [21]:
idx = 0
sample = data[idx, :, :].cpu().numpy()
gen_sample = gen_data[idx, :, :].cpu().numpy()
plt.plot(sample[0, :])
plt.plot(gen_sample[0, :])
print(sample[:5, :5] - gen_sample[:5, :5])
print(f"MSE = {np.power((sample - gen_sample), 2).sum() / 2}")

NameError: name 'plt' is not defined

In [ ]:
idx = 1
sample = data[idx, :, :].cpu().numpy()
gen_sample = gen_data[idx, :, :].cpu().numpy()
plt.plot(sample[0, :])
plt.plot(gen_sample[0, :])
print(sample[:5, :5] - gen_sample[:5, :5])
print(f"MSE = {np.power((sample - gen_sample), 2).sum() / 2}")

In [ ]:
err = np.zeros(targets.shape)
for i, target in enumerate(targets):
    err[i] = -loss_desc['logp'][i]
    print(f"Target = {target}, MSE = {err[i]:.3f}")
    if i > 10:
        break

In [ ]:
plt.scatter(targets, err)

Show the data plots.

In [ ]:
show_plot(sample)

In [ ]:
show_plot(gen_sample)

Let's score the success of the recreation and look for outliers.

In [ ]:
scores = score(test_dl, model, criterion)

In [ ]:
test_means = pd.DataFrame()
for (name, cls), item in scores.items():
    test_means.loc[name, cls] = np.array(item).mean()

print("###################### TEST MEANS #####################")
print(test_means)

In [ ]:
val_means = pd.DataFrame()
for (name, cls), item in score(val_dl, model, criterion).items():
    val_means.loc[name, cls] = np.array(item).mean()

print("###################### VAL MEANS #####################")
print(val_means)

In [ ]:
train_means = pd.DataFrame()
for (name, cls), item in score(train_dl, model, criterion).items():
    train_means.loc[name, cls] = np.array(item).mean()

print("###################### TRAIN MEANS #####################")
print(train_means)

Let's calculate AUC to judge performance.

In [ ]:
scores.keys()

In [ ]:
print(len(scores[('error', 'norm')]), len(scores[('error', 'fail')]))

In [ ]:
# simple definition of the threshold as mean between sets
threshold = (np.mean(scores['error', 'fail']) +
             np.mean(scores['error', 'norm'])) / 2
print('mean threshold:', threshold)
# maximum 95% percentile of normal as threshold
# threshold = np.percentile(scores['error', 'norm'], 95)
# print('95th percentile threshold:', threshold)

In [ ]:
t_scores = scores.copy()
t_scores[('error', 'fail')] = (t_scores[('error', 'fail')] > threshold)
t_scores[('error', 'norm')] = (t_scores[('error', 'norm')] > threshold)

t_score = []
t_score.extend(t_scores[('error', 'fail')])
t_score.extend(t_scores[('error', 'norm')])
t_score = np.array(t_score)

y_true = []
y_true.extend([True] * len(t_scores[('error', 'fail')]))
y_true.extend([False] * len(t_scores[('error', 'norm')]))
y_true = np.array(y_true)

In [ ]:
t_corr = t_score == y_true
t_corr.sum() / len(t_corr)

In [ ]:
f1_score(y_true, t_score)

In [ ]:
auc_scores = auc_score(test_dl, t_scores)
auc_scores

In [ ]:
np.mean(scores['error', 'norm'])

In [ ]:
normaly = sorted(scores[('error', 'norm')])
anomaly = sorted(scores[('error', 'fail')])
plt.plot(normaly, label='Normal MSE')
plt.plot(anomaly, label='Anomaly MSE')
plt.legend()

In [ ]:
plt.hist(normaly, label='Normal MSE', bins=100, alpha=0.5)
plt.hist(anomaly, label='Anomaly MSE', bins=100, alpha=0.5)
plt.legend()

In [ ]:
normalx = np.random.rand(len(normaly))
anomalx = np.random.rand(len(anomaly))

In [ ]:
plt.scatter(normaly, normalx, label='Normal MSE (test)', alpha=0.5)
plt.scatter(anomaly, anomalx, label='Anomaly MSE (test)', alpha=0.5)
plt.plot([threshold, threshold], [0, 1], 'k-')
plt.legend()

Try fitting a normal distribution to the validation MSE to define a threshold.

In [ ]:
val_scores = score(val_dl, model, criterion)

In [ ]:
val_scores.keys()

In [ ]:
val_norm = sorted(val_scores[('error', 'norm')])
print(val_norm[:5])
mean = np.mean(val_norm)
std = np.std(val_norm)
print('mean={:.2f}, stdev={:.2f}'.format(mean, std))
plt.plot(val_norm, label= 'Normal MSE (validation)')
plt.plot([0, len(val_norm)], [mean, mean], label='Mean')
plt.legend()

Implement PCA to visualize the latent space.

In [ ]:
latents, targets = compute_latent(test_dl, model)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
lat_pca = pca.fit_transform(latents)

In [ ]:
lat_pca.shape

In [ ]:
fail_mask = targets == 0
norm_mask = targets == 1
plt.scatter(lat_pca[fail_mask, 0], lat_pca[fail_mask, 1],
            c='orange', label='fault')
plt.scatter(lat_pca[norm_mask, 0], lat_pca[norm_mask, 1],
            c='skyblue', label='normal')
plt.legend()

In [ ]:
pca.explained_variance_ratio_

Cluster the latent space into kmeans clusters.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(3)

In [ ]:
lat_k = kmeans.fit_transform(latents)

In [ ]:
lat_k.shape

In [ ]:
plt.scatter(lat_pca[:, 0], lat_pca[:, 1], c=kmeans.labels_)

Try adding the error terms to the latent features.

In [ ]:
latents, kl, error, targets = compute_latent_and_loss(test_dl, model, criterion)

In [ ]:
print(latents.shape, kl.shape, error.shape, targets.shape)

In [ ]:
features = np.hstack([latents, kl[:, None], error[:, None]])
features.shape

In [ ]:
pca = PCA(n_components=2)
lat_pca = pca.fit_transform(features)
lat_pca.shape

In [ ]:
plt.scatter(lat_pca[fail_mask, 0], lat_pca[fail_mask, 1],
            c='orange', label='fault')
plt.scatter(lat_pca[norm_mask, 0], lat_pca[norm_mask, 1],
            c='skyblue', label='normal')
plt.legend()

In [ ]:
pca.explained_variance_ratio_

Compare the pca plots for validation and training sets.

In [ ]:
latents, targets = compute_latent(train_dl, model)
pca = PCA(n_components=2)
lat_pca = pca.fit_transform(latents)
plt.scatter(lat_pca[:, 0], lat_pca[:, 1],
            c='skyblue', label='normal')
plt.legend()

In [ ]:
latents, targets = compute_latent(val_dl, model)
pca = PCA(n_components=2)
lat_pca = pca.fit_transform(latents)
plt.scatter(lat_pca[:, 0], lat_pca[:, 1],
            c='skyblue', label='normal')
plt.legend()

Combine the validation plot with the test plot to understand global structure.

In [ ]:
latents, targets = compute_latent(train_dl, model)
pca = PCA(n_components=2)
train_pca = pca.fit_transform(latents)

In [ ]:
latents, targets = compute_latent(val_dl, model)
val_pca = pca.transform(latents)

In [ ]:
latents, targets = compute_latent(test_dl, model)
test_pca = pca.transform(latents)
fail_mask = targets == 0
norm_mask = targets == 1

In [ ]:
plt.scatter(test_pca[fail_mask, 0], test_pca[fail_mask, 1],
            c='orange', label=desc + ' fault')
plt.scatter(test_pca[norm_mask, 0], test_pca[norm_mask, 1],
            c='indigo', label='normal-test')
plt.scatter(val_pca[:, 0], val_pca[:, 1],
            c='midnightblue', label='normal-val')
plt.scatter(train_pca[:, 0], train_pca[:, 1],
            c='darkslateblue', label='normal-train')
plt.legend()

In [ ]:
plt.scatter(test_pca[norm_mask, 0], test_pca[norm_mask, 1],
            c='indigo', label='normal-test')
plt.scatter(val_pca[:, 0], val_pca[:, 1],
            c='midnightblue', label='normal-val')
plt.scatter(train_pca[:, 0], train_pca[:, 1],
            c='darkslateblue', label='normal-train')
plt.scatter(test_pca[fail_mask, 0], test_pca[fail_mask, 1],
            c='orange', label=desc + ' fault')
plt.legend()